In [5]:
%pip -q install google-genai

In [6]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [7]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [8]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

In [9]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [10]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [11]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [12]:
##########################################
# --- Agente 1: Agente de viagem [Turismo] --- #
##########################################
def agente_turismo(destino):

    turismo = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um agente de viagens responsável por fornecer um panorama geral e envolvente do destino da viagem para o cliente.
        Seu objetivo é despertar o interesse e fornecer informações essenciais antes que o roteiro detalhado seja apresentado.

        Com base no destino fornecido pelo cliente, realize uma pesquisa utilizando a ferramenta de busca do Google (google_search) para apresentar um breve resumo e informações detalhadas em cinco categorias principais:

1. Panorama Geral:

Breve Resumo Histórico: Apresente um resumo conciso da história do país ou região, destacando eventos importantes e influências culturais.
Principais Curiosidades: Compartilhe algumas curiosidades interessantes sobre o local, como costumes únicos, fatos notáveis ou peculiaridades culturais.
Notícias Recentes Relevantes: Mencione brevemente notícias recentes que possam ser de interesse para um turista (ex: eventos culturais significativos, novas atrações, informações sobre segurança ou infraestrutura turística).

2. Cultura + Principais Pontos Turísticos/Locais Históricos:

Cultura: Descreva brevemente os aspectos culturais mais importantes do destino (ex: idioma, religião predominante, artes, música, dança, festivais).
Principais Pontos Turísticos: Liste e descreva brevemente 3-5 dos pontos turísticos mais icônicos e imperdíveis do destino.
Locais Históricos: Mencione 2-3 locais históricos relevantes, explicando sua importância e o que os torna interessantes para visitar.

3. Gastronomia + Principais Restaurantes com Comida Local:

Gastronomia: Apresente uma visão geral da culinária local, mencionando ingredientes típicos, pratos tradicionais e influências.
Principais Restaurantes com Comida Local: Sugira 2-3 restaurantes bem avaliados que ofereçam uma autêntica experiência da culinária local, mencionando brevemente seus destaques.

4. Hospedagem e Transporte + Dicas:

Hospedagem: Forneça dicas sobre as melhores regiões para se hospedar, considerando a proximidade dos principais pontos turísticos, segurança e ambiente geral (ex: centro histórico, áreas mais tranquilas, bairros com vida noturna).
Transporte: Ofereça informações sobre as principais opções de transporte disponíveis no destino (transporte público, táxis, aplicativos, aluguel de carro) e dicas sobre qual pode ser mais conveniente para diferentes situações.

5. Atividades/Experiências + Dicas:

Esportes e Atividades ao Ar Livre: Se aplicável ao destino, sugira locais ou atividades para a prática de esportes (ex: trilhas, praias para esportes aquáticos, parques) e outras experiências ao ar livre (ex: passeios de barco, observação de vida selvagem).
Experiências Locais: Incentive o cliente a vivenciar a cultura local, sugerindo atividades como visitar mercados tradicionais, assistir a apresentações culturais, participar de workshops ou interagir com a comunidade local.

Formato da Resposta:

Apresente o panorama geral de forma clara, concisa e envolvente. Utilize títulos e subtítulos para cada categoria.
Seja informativo e busque despertar a curiosidade do cliente sobre o destino.
        """,
        description="Agente que crie um resumo e panorama geral sobre o destino da viagem",
        tools=[google_search]
    )

    entrada_do_agente_turismo = f"Destino: {destino}"

    resumo = call_agent(turismo, entrada_do_agente_turismo)
    return resumo

In [13]:
################################################
# --- Agente 2: Agente de viagem [financeiro] --- #
################################################


def agente_financeiro(destino, cidade_partida, epoca, duracao, num_pessoas, experiencia, interesses, orcamento):
    financeiro = Agent(
        name="agente_financeiro",
        model="gemini-2.0-flash",
        instruction="""
        Você é um agente de viagens especializado em fornecer um planejamento financeiro detalhado para os clientes.
        Sua tarefa é estimar os custos totais de uma viagem com base no destino e demais informações fornecidas pelo cliente.

        Com base no destino fornecido pelo cliente e outras informações relevantes, realize uma pesquisa utilizando a ferramenta de busca do Google (google_search) para buscar os seguintes custos:

1 Valor das Passagens Aéreas:

Estime o custo médio das passagens aéreas de uma cidade de partida comum para o destino (se a cidade de partida não for fornecida, mencione isso na estimativa).
Apresente o valor por pessoa e o valor total para o número de viajantes.
Mencione que os preços podem variar dependendo da época da viagem, antecedência da compra e companhia aérea.

2 Valor da Estadia (Hospedagem):

Estime o custo médio da hospedagem por noite no destino, considerando diferentes categorias (econômico, padrão, luxo).
Calcule o custo total estimado para a duração da viagem e o número de viajantes.
Mencione que os preços podem variar dependendo da localização, tipo de acomodação e época da viagem.

3 Valor da Alimentação:

Estime o custo médio diário por pessoa para alimentação, considerando opções econômicas e restaurantes de nível médio.
Calcule o custo total estimado para a duração da viagem e o número de viajantes.
Se o cliente mencionar preferências específicas de restaurantes, tente encontrar uma faixa de preço média para esses tipos de estabelecimentos no destino.

4 Valor do Transporte:

Estime os custos médios para se locomover no destino, considerando transporte público e aplicativos de transporte.
Apresente uma estimativa diária ou para a duração da viagem (se possível).
Se o cliente mencionar interesse em alugar um carro, inclua uma estimativa do custo médio da diária e possíveis custos adicionais (seguro, combustível).

5 Valor dos Passeios:

Com base nos tipos de experiências e interesses mencionados pelo cliente, liste alguns passeios populares no destino e seus custos médios.
Apresente um valor total estimado para os passeios sugeridos.

6 Caixa Extra para Compras e Emergências:

Sugira uma porcentagem (por exemplo, 10-15%) do custo total estimado da viagem como um caixa extra para imprevistos ou compras.
Apresente o valor sugerido em moeda local (se identificada) e em Reais (se possível).

Formato da Resposta:

Apresente o planejamento financeiro de forma clara e organizada, com subtotais para cada categoria e um valor total estimado da viagem.


        """,
        description="Agente que faz o planejamento financeiro de viagens",
        tools=[google_search]
    )
    entrada_do_agente_financeiro = f"Destino: {destino}"\
    f"Partida de: {cidade_partida}"\
    f"Época: {epoca}"\
    f"Duração: {duracao}"\
    f"Pessoas: {num_pessoas}"\
    f"Experiência: {experiencia}"\
    f"Interesses: {interesses}"\
    f"Orçamento: {orcamento}"

    planejamento = call_agent(financeiro, entrada_do_agente_financeiro)
    return planejamento

    #Utilize a moeda local do destino (se identificada) e, se possível, forneça uma conversão aproximada para Reais (BRL).

In [14]:
######################################
# --- Agente 3: Agente de viagem [Roteiro] --- #
######################################
def agente_roteiro(destino, cidade_partida, epoca, duracao, num_pessoas, experiencia, interesses, orcamento, planejamento):
    roteiro = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um agente de viagens especializado em criar roteiros personalizados e detalhados para os clientes.
            Sua tarefa é desenvolver um plano de viagem completo, levando em consideração diversas informações fornecidas pelo cliente para garantir uma experiência memorável e alinhada com suas expectativas.

            Com base nas informações fornecidas pelo cliente, crie um roteiro detalhado para a viagem, incluindo:

1 Visão Geral do Roteiro:

Um breve resumo da proposta do roteiro, destacando como ele atende às preferências e interesses do cliente.
Mencione o ritmo geral do roteiro (intenso, equilibrado, relaxado).

2 Cronograma Diário Detalhado: Para cada dia da viagem, inclua:

Manhã: Sugestão de atividade ou ponto turístico a ser visitado.
Almoço: Sugestão de local para almoçar que esteja alinhado com as preferências (se fornecidas) ou seja conveniente para a atividade da manhã. Mencione o tipo de culinária e faixa de preço (se possível).
Tarde: Sugestão de atividade ou ponto turístico a ser visitado.
Noite: Sugestão de atividade (jantar, evento cultural, vida noturna, tempo livre). Mencione o tipo de culinária e faixa de preço (se possível) para o jantar.
Observações: Informações adicionais relevantes, como tempo estimado de deslocamento entre as atividades, necessidade de reserva, melhor forma de chegar ao local (de acordo com as preferências de transporte), e dicas extras.

3 Considerações Especiais:

Para a época da viagem: Mencione se há eventos especiais, clima típico ou outras considerações importantes para o período.
Em relação ao orçamento: Se o orçamento for apertado, sugira opções de atividades gratuitas ou de baixo custo e restaurantes mais acessíveis. Se o orçamento for mais flexível, inclua opções mais exclusivas.
Atendendo às preferências de experiências e interesses: Certifique-se de que as atividades propostas estejam diretamente relacionadas aos interesses e ao tipo de experiência desejada pelo cliente.
Dias Livres (se aplicável): Se houver dias livres no roteiro, forneça algumas sugestões opcionais de atividades ou áreas para explorar com base nos interesses do cliente.

4 Informações Práticas:

Transporte: Breves orientações sobre como se locomover no destino, com base nas preferências do cliente e nas sugestões do roteiro.
Alimentação: Dicas gerais sobre a culinária local e sugestões de tipos de restaurantes a experimentar.

Formato da Resposta:

Apresente o roteiro de forma clara, organizada e fácil de seguir.
Utilize títulos e subtítulos para cada dia e atividade.
Seja conciso, mas forneça informações suficientes para que o cliente tenha uma boa compreensão do plano.




            """,
        description="Agente que faz o roteiro de viagens"
    )
    entrada_do_agente_roteiro = f"Destino: {destino}"\
    f"Partida de: {cidade_partida}"\
    f"Época: {epoca}"\
    f"Duração: {duracao}"\
    f"Pessoas: {num_pessoas}"\
    f"Experiência: {experiencia}"\
    f"Interesses: {interesses}"\
    f"Orçamento: {orcamento}"

    # Executa o agente
    guia_viagem = call_agent(roteiro, entrada_do_agente_roteiro)
    return guia_viagem

In [15]:
print("🚀 Iniciando o Sistema de Planejamento de Viagens Agentes 🚀")

# --- Coleta de informações do usuário ---
destino = input("Por favor, digite o DESTINO para onde quer viajar: ")
cidade_partida = input("De qual cidade você irá embarcar?: ")
epoca = input("Qual época (ou data) está planejando viajar?: ")
duracao = input("Qual a duração da viagem (em dias)?: ")
num_pessoas = input("Quantas pessoas irão viajar?: ")
experiencia = input("""Qual o principal tipo de experiência que você busca em suas férias? \n
- imersão cultural\n
- foco na gastronomia\n
- descanso/relaxamento\n
- aventura emocionante\n
Sinta-se à vontade para descrever o que mais te atrai: """)
interesses = input("""Você tem algum interesse ou hobby específico que gostaria de incluir na sua viagem? \n
- corrida de rua\n
- shoppings\n
- vinícolas\n
- livros\n
Compartilhe seus interesses: """)
orcamento = input("Qual a sua estimativa de orçamento total para esta viagem (em reais - R$)?: ")
"\n"

# Inserir lógica do sistema de agentes ################################################
if not destino:
    print("Você esqueceu de digitar o DESTINO!")
else:
    print(f"Maravilha! Vamos criar o seu plano de viagem para {destino}")

    resumo = agente_turismo(destino)
    print("\n--- 📝 Resultado do Agente 1 (Resumo) ---\n")
    display(to_markdown(resumo))
    print("--------------------------------------------------------------")

    planejamento = agente_financeiro(destino, cidade_partida, epoca, duracao, num_pessoas, experiencia, interesses, orcamento)
    print("\n--- 📝 Resultado do Agente 2 (Financeiro) ---\n")
    display(to_markdown(planejamento))
    print("--------------------------------------------------------------")

    guia_viagem = agente_roteiro(destino, cidade_partida, epoca, duracao, num_pessoas, experiencia, interesses, orcamento, planejamento)
    print("\n--- 📝 Resultado do Agente 3 (Roteiro) ---\n")
    display(to_markdown(guia_viagem))

🚀 Iniciando o Sistema de Planejamento de Viagens Agentes 🚀
Por favor, digite o DESTINO para onde quer viajar: madri
De qual cidade você irá embarcar?: sao apulo
Qual época (ou data) está planejando viajar?: abril 2026
Qual a duração da viagem (em dias)?: 10 dias
Quantas pessoas irão viajar?: 1 pessoa
Qual o principal tipo de experiência que você busca em suas férias? 

- imersão cultural

- foco na gastronomia

- descanso/relaxamento

- aventura emocionante

Sinta-se à vontade para descrever o que mais te atrai: cultura
Você tem algum interesse ou hobby específico que gostaria de incluir na sua viagem? 

- corrida de rua

- shoppings

- vinícolas

- livros

Compartilhe seus interesses: compras
Qual a sua estimativa de orçamento total para esta viagem (em reais - R$)?: 
10000
Maravilha! Vamos criar o seu plano de viagem para madri

--- 📝 Resultado do Agente 1 (Resumo) ---



> Olá! Madri é um destino incrível, cheio de história, cultura vibrante e gastronomia deliciosa. Aqui está um panorama geral para despertar o seu interesse:
> 
> 
> ## 1. Panorama Geral:
> 
> *   **Breve Resumo Histórico**: Madri tem suas origens em um forte mouro do século IX, chamado "Mayrit". A cidade ganhou importância quando Filipe II transferiu a corte para lá em 1561, tornando-se a capital da Espanha. Ao longo dos séculos, Madri passou por transformações urbanísticas e eventos históricos marcantes, como a Guerra da Sucessão e a Guerra Civil.
> *   **Principais Curiosidades:** Os habitantes de Madri são chamados de "gatos". A cidade abriga o restaurante mais antigo do mundo, o Botín, fundado em 1725. O Palácio Real de Madri é o maior palácio da Europa Ocidental. Madri é a capital mais alta da Europa, situada a 650 metros acima do nível do mar.
> *   **Notícias Recentes Relevantes:** Em fevereiro de 2025, Madri continua a ser um destino vibrante com uma rica oferta cultural e gastronômica. Há sempre novos espaços culturais alternativos e mercados surgindo.
> 
> ## 2. Cultura + Principais Pontos Turísticos/Locais Históricos:
> 
> *   **Cultura:** A cultura madrilenha é caracterizada pela alegria de viver, arte e uma rica gastronomia. O idioma é o espanhol, e a cidade possui uma forte tradição em artes, música e dança, especialmente o flamenco. Madri é também conhecida por seus festivais e eventos culturais ao longo do ano.
> *   **Principais Pontos Turísticos:**
>     *   **Museu do Prado:** Um dos museus de arte mais importantes do mundo, com obras de mestres como Velázquez, Goya e El Greco.
>     *   **Parque El Retiro:** Um vasto parque urbano com jardins, monumentos, um lago e áreas de recreação.
>     *   **Palácio Real:** A residência oficial da família real espanhola, com salas ricamente decoradas e uma impressionante coleção de arte.
>     *   **Plaza Mayor:** Uma praça histórica cercada por edifícios icônicos, palco de eventos e celebrações.
>     *   **Gran Vía:** Uma avenida movimentada com arquitetura impressionante, lojas e teatros.
> *   **Locais Históricos:**
>     *   **Templo de Debod:** Um templo egípcio do século II a.C., presenteado à Espanha pelo Egito, com vistas panorâmicas da cidade.
>     *   **Plaza de la Villa:** Uma praça com edifícios históricos que remontam à época medieval de Madri.
> 
> ## 3. Gastronomia + Principais Restaurantes com Comida Local:
> 
> *   **Gastronomia:** A culinária de Madri é uma celebração de sabores, com pratos tradicionais como o cocido madrileño (um guisado de grão-de-bico, vegetais e carnes), callos a la madrileña (dobradinha) e sanduíche de lula. As tapas são uma parte essencial da experiência gastronômica madrilenha.
> *   **Principais Restaurantes com Comida Local:**
>     *   **Restaurante Botín:** O restaurante mais antigo do mundo, famoso por seus assados, especialmente o leitão.
>     *   **La Bola Taberna:** Um restaurante familiar que serve um autêntico cocido madrileño preparado em potes de barro individuais.
>     *   **Lhardy:** Um restaurante histórico que serve pratos da culinária madrilenha e espanhola desde 1839.
> 
> ## 4. Hospedagem e Transporte + Dicas:
> 
> *   **Hospedagem:** As melhores regiões para se hospedar em Madri incluem o centro da cidade (Sol, Gran Vía), o Barrio de las Letras (para uma experiência cultural), La Latina (para um ambiente boêmio) e Malasaña (para vida noturna).
> *   **Transporte:** Madri possui um sistema de transporte público eficiente, incluindo metrô, ônibus e trem. O metrô é a maneira mais rápida de se locomover. Caminhar é uma ótima opção, pois Madri é uma cidade amigável para pedestres. Para usar o transporte público, você pode adquirir a Tarjeta Multi, um cartão recarregável.
> ## 5. Atividades/Experiências + Dicas:
> 
> *   **Esportes e Atividades ao Ar Livre:** O Parque El Retiro oferece atividades como passeios de barco e caminhadas. O Madrid Río, um parque linear ao longo do rio Manzanares, é ideal para ciclismo e corrida. Nos arredores de Madri, a Serra de Guadarrama é um destino para caminhantes e amantes de esportes ao ar livre.
> *   **Experiências Locais:** Visite mercados tradicionais como o Mercado de San Miguel e El Rastro. Assista a um show de flamenco. Explore os bairros históricos e suas praças. Desfrute de um chocolate quente na Chocolateria San Ginés.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Financeiro) ---



> Olá! Com base nas informações fornecidas, preparei um planejamento financeiro detalhado para sua viagem a Madri em abril de 2026, com foco em cultura e compras. Aqui está uma estimativa dos custos totais, considerando sua partida de São Paulo, duração de 10 dias e orçamento de R$10.000:
> 
> 
> ## Planejamento Financeiro Detalhado para Viagem a Madri (Abril 2026)
> 
> **1. Passagens Aéreas:**
> 
> *   **Estimativa:** R$ 3.000 - R$ 5.000 (ida e volta).
> *   **Observação:** Os preços das passagens aéreas de São Paulo para Madri podem variar significativamente dependendo da antecedência da compra, da companhia aérea e da época da viagem. Abril é um mês de alta temporada, então os preços podem ser mais elevados.
> *   **Total:** R$ 3.000 - R$ 5.000 (por pessoa).
> 
> **2. Hospedagem:**
> 
> *   **Estimativa:**
>     *   Econômico (hostel/hotel 2-3 estrelas): R$ 250 - R$ 400 por noite.
>     *   Padrão (hotel 3-4 estrelas): R$ 500 - R$ 800 por noite.
>     *   Luxo (hotel 5 estrelas): Acima de R$ 1.500 por noite.
> *   **Total (10 noites):**
>     *   Econômico: R$ 2.500 - R$ 4.000.
>     *   Padrão: R$ 5.000 - R$ 8.000.
>     *   Luxo: Acima de R$ 15.000.
> 
> **3. Alimentação:**
> 
> *   **Estimativa:**
>     *   Econômico: €40 por dia (R$220,00).
>     *   Restaurantes nível médio: €82 por dia (R$451,00).
> *   **Total (10 dias):**
>     *   Econômico: R$ 2.200.
>     *   Restaurantes nível médio: R$ 4.510.
> 
> **4. Transporte:**
> 
> *   **Estimativa:**
>     *   Transporte público (Metrô e ônibus): O bilhete que vale por 10 viagens custa 14,20€ (R$78,00). Bilhete único: 1,50€ (R$8,25).
>     *   Aplicativos de transporte: Varia muito dependendo da distância e demanda.
> *   **Total (10 dias):**
>     *   Transporte público: R$ 400 (estimativa).
>     *   Aplicativos de transporte: R$ 500 (estimativa).
> 
> **5. Passeios:**
> 
> *   **Sugestões:**
>     *   Museu do Prado: 15€ (R$82,50).
>     *   Palácio Real: 13€ (R$71,50).
>     *   Estádio Santiago Bernabéu: 15€ (R$82,50).
>     *   Show de Flamenco: A partir de 25€ (R$137,50).
> *   **Total:** R$ 500 (estimativa).
> 
> **6. Compras:**
> 
> *   **Estimativa:** Depende muito do seu estilo de compra. Se você pretende fazer compras, reserve uma parte significativa do seu orçamento.
> *   **Sugestão:** Defina um valor máximo para gastos com compras antes da viagem para evitar estouros no orçamento.
> 
> **7. Caixa Extra para Imprevistos:**
> 
> *   **Sugestão:** 15% do custo total estimado da viagem.
> *   **Estimativa:** R$ 1.500 (em Reais).
> 
> **Resumo dos Custos (Estimativa)**
> 
> Considerando um estilo de viagem econômico a médio:
> 
> *   Passagens Aéreas: R$ 3.000 - R$ 5.000
> *   Hospedagem: R$ 2.500 - R$ 8.000
> *   Alimentação: R$ 2.200 - R$ 4.510
> *   Transporte: R$ 400 - R$ 500
> *   Passeios: R$ 500
> *   Caixa Extra: R$ 1.500
> *   **Total Estimado:** R$ 10.100 - R$ 20.010
> 
> **Observações:**
> 
> *   Esta é uma estimativa, e os custos reais podem variar.
> *   Os preços em euros foram convertidos para reais utilizando uma taxa de câmbio de 1 EUR = R$ 5,50.
> *   Recomenda-se monitorar os preços das passagens aéreas e hospedagem com antecedência e reservar o quanto antes para garantir melhores tarifas.
> *   Abril é um mês de alta temporada em Madri, então é importante planejar e reservar com antecedência para evitar preços mais altos e disponibilidade limitada.
> *   Considere a compra de um seguro de viagem, que é obrigatório para entrar na Espanha e pode te proteger contra imprevistos.
> 
> Espero que este planejamento financeiro detalhado te ajude a organizar sua viagem a Madri!


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Roteiro) ---



> ## Roteiro Detalhado de 10 Dias em Madri (Abril de 2026)
> 
> **Visão Geral do Roteiro:**
> 
> Este roteiro foi cuidadosamente elaborado para proporcionar uma imersão na rica cultura de Madri, com foco em experiências autênticas, arte, história e, claro, tempo para compras. O ritmo é equilibrado, permitindo que você explore a cidade com calma e aproveite cada momento.
> 
> **Cronograma Diário Detalhado:**
> 
> **Dia 1: Chegada em Madri e Boas-Vindas**
> 
> *   **Manhã:** Chegada ao Aeroporto Adolfo Suárez Madrid-Barajas (MAD). Traslado para o hotel (sugestão: utilize o serviço de trem expresso ou táxi/aplicativo). Check-in e tempo para descanso.
> *   **Almoço:** Restaurante "Sobrino de Botín" (cozinha tradicional espanhola, faixa de preço média-alta), o restaurante mais antigo do mundo, para uma experiência gastronômica histórica.
> *   **Tarde:** Passeio relaxante pelo Parque del Retiro, o pulmão verde de Madri. Explore o lago, o Palácio de Cristal e os jardins.
> *   **Noite:** Jantar no Mercado de San Miguel (tapas e vinhos, faixa de preço média), um mercado gourmet com diversas opções de petiscos espanhóis.
> 
> **Dia 2: Arte e História no Triângulo Dourado**
> 
> *   **Manhã:** Visita ao Museu do Prado, um dos mais importantes museus de arte do mundo. Foque nas obras de Goya, Velázquez e El Greco.
> *   **Almoço:** "Taberna La Carmencita" (cozinha espanhola tradicional, faixa de preço média), um restaurante histórico com pratos clássicos.
> *   **Tarde:** Exploração do Museu Reina Sofía, com foco na arte moderna e contemporânea espanhola, incluindo a famosa obra "Guernica" de Picasso.
> *   **Noite:** Show de flamenco no Corral de la Morería (jantar e show, faixa de preço alta), uma experiência autêntica da cultura espanhola.
> 
> **Dia 3: Palácio Real e Compras na Gran Vía**
> 
> *   **Manhã:** Visita ao Palácio Real de Madri, a residência oficial da família real espanhola. Explore os salões, a coleção de arte e os jardins.
> *   **Almoço:** "Mercado de San Antón" (opções variadas, faixa de preço média), um mercado moderno com restaurantes e bares no terraço.
> *   **Tarde:** Compras na Gran Vía, a principal avenida de Madri, com lojas de departamento, boutiques de grife e cinemas.
> *   **Noite:** Jantar no "Rosi La Loca" (cozinha internacional com toques espanhóis, faixa de preço média), um restaurante moderno e animado.
> 
> **Dia 4: Toledo, a Cidade Imperial**
> 
> *   **Dia inteiro:** Excursão de um dia a Toledo, uma cidade histórica Patrimônio Mundial da UNESCO. Visite a Catedral de Toledo, a Sinagoga de Santa María la Blanca e admire a vista panorâmica da cidade.
> *   **Almoço:** Restaurante em Toledo (cozinha tradicional da região, faixa de preço média), experimente pratos como o "cozido toledano".
> *   **Noite:** Retorno a Madri e jantar livre (sugestão: explore um bairro diferente como La Latina).
> 
> **Dia 5: El Rastro e o Templo de Debod**
> 
> *   **Manhã:** Visita ao El Rastro, o maior mercado de rua de Madri (somente aos domingos). Explore as barracas de antiguidades, roupas, artesanato e comida.
> *   **Almoço:** Tapas em La Latina (faixa de preço baixa a média), aproveite a atmosfera animada do bairro.
> *   **Tarde:** Visita ao Templo de Debod, um templo egípcio doado à Espanha. Admire o pôr do sol com uma vista panorâmica da cidade.
> *   **Noite:** Jantar no "Lateral" (tapas e petiscos, faixa de preço média), um restaurante popular com diversas opções de tapas.
> 
> **Dia 6: Dia Livre para Exploração**
> 
> *   **Dia inteiro:** Dia livre para explorar Madri no seu próprio ritmo. Algumas sugestões:
>     *   Visitar o Estádio Santiago Bernabéu (para fãs de futebol).
>     *   Explorar o bairro de Malasaña, conhecido pela sua atmosfera boêmia e lojas vintage.
>     *   Fazer um tour gastronômico para degustar os sabores de Madri.
> *   **Almoço e Jantar:** Livre, explore as opções gastronômicas da cidade.
> 
> **Dia 7: Segóvia e Aqueduto Romano**
> 
> *   **Dia inteiro:** Excursão de um dia a Segóvia, outra cidade histórica com um impressionante aqueduto romano. Visite o Alcázar de Segóvia, a Catedral e explore as ruas medievais.
> *   **Almoço:** Restaurante em Segóvia (cozinha tradicional da região, faixa de preço média), experimente o "cochinillo asado" (leitão assado).
> *   **Noite:** Retorno a Madri e jantar no "La Barraca" (paella, faixa de preço média), um restaurante especializado em paella.
> 
> **Dia 8: Compras de Luxo e Jardim Botânico**
> 
> *   **Manhã:** Compras no bairro de Salamanca, conhecido pelas suas boutiques de luxo e marcas internacionais.
> *   **Almoço:** Restaurante no bairro de Salamanca (opções variadas, faixa de preço média a alta).
> *   **Tarde:** Visita ao Real Jardín Botánico, um oásis de tranquilidade com diversas espécies de plantas.
> *   **Noite:** Jantar no "Ten con Ten" (cozinha espanhola moderna, faixa de preço alta), um restaurante elegante e sofisticado.
> 
> **Dia 9: Mercado de San Fernando e Arte Urbana**
> 
> *   **Manhã:** Visita ao Mercado de San Fernando, um mercado local com produtos frescos, restaurantes e lojas de artesanato.
> *   **Almoço:** Restaurante no Mercado de San Fernando (opções variadas, faixa de preço baixa a média).
> *   **Tarde:** Exploração do bairro de Lavapiés, conhecido pela sua arte urbana, atmosfera multicultural e lojas alternativas.
> *   **Noite:** Jantar no "El Sur" (tapas e petiscos, faixa de preço média), um restaurante popular com um ambiente acolhedor.
> 
> **Dia 10: Despedida de Madri**
> 
> *   **Manhã:** Últimas compras ou visita a algum ponto turístico que ficou faltando.
> *   **Almoço:** Livre (sugestão: aproveite para experimentar um último prato típico).
> *   **Tarde:** Traslado para o Aeroporto Adolfo Suárez Madrid-Barajas (MAD) e embarque para São Paulo.
> 
> **Considerações Especiais:**
> 
> *   **Época da Viagem (Abril de 2026):** Abril é uma ótima época para visitar Madri, com temperaturas amenas e agradáveis. Verifique se há algum evento especial ou festival acontecendo durante a sua estadia.
> *   **Orçamento:** Com um orçamento de 10.000 euros, você pode desfrutar de uma viagem confortável e com algumas experiências mais exclusivas.
> *   **Preferências de Experiências e Interesses:** O roteiro foi elaborado para atender aos seus interesses em cultura e compras, com foco em museus, locais históricos, mercados e áreas de compras variadas.
> 
> **Informações Práticas:**
> 
> *   **Transporte:** Madri possui um excelente sistema de transporte público, incluindo metrô, ônibus e trem. Caminhar também é uma ótima maneira de explorar a cidade. Considere adquirir um cartão de transporte para facilitar o uso do transporte público.
> *   **Alimentação:** A culinária espanhola é rica e variada. Experimente tapas, paella, jamón ibérico, churros com chocolate e vinhos locais. Explore os mercados e restaurantes de diferentes bairros para descobrir novos sabores.
> 
> Espero que este roteiro detalhado ajude você a planejar uma viagem inesquecível a Madri!
